# Experiment 02  -  RC Transients

Date:

Name:

Partner's name:

In [ ]:
from common import *

## Place a heading here to indicate what you are working on

Enter notes here as you work through the experiment. Begin with a brief statement about what you will be working on, and why. 

## You might need another heading here

As you progress through the lab, keep adding notes, and include headings to help keep things organized. You can also include calculatino blocks to determine parameters along the way. We will provide much less structure in this notebook, so it is up to you to take notes as you go along. We will be checking your work during the lab.

## Measurements of Resistance and Capacitance

Record the measured values of R and C for the components that you are using in your circuit. To estimate the uncertainties in these values, you will need to consult the manual for your DMM. Use a code cell to input the values and calculate $\tau$ as well as propagate the uncertainties.

## Upload Data Files to Your Jupyter Account and Pack the Data

Below, we have provided a more compact version of the code you used last week to examine your data and then do some averaging to pack it. Once you have saved a set of data from the oscilloscope, and uploaded it to your Jupyter account, the entire code runs in one cell. You will probably need to run it several times until you are satisfied with the final output. 

The key pieces of code that you will need to alter are indicated by notes in Red, and include:
- the name for your uploaded raw data file
- the zoomed in flat range you use to examine the noise and measure the standard deviation
- the number of points you want to average when packing the data
- a new index range that you can use to trim unwanted data at the start or end of the time range
- the filename you will use to save your packed and trimmed data 

In [ ]:
gopts = VOLTAGE_VERSUS_TIME_GRAPH_OPTIONS
raw = load_raw_data('RCdecayVC3.csv', plot=True, graphing_options=gopts)

In [ ]:
noise = isolate_noise(raw, indices_range=(70500,71000), y_range=(0.01,0.04),
                      plot=True, graphing_options=gopts)

In [ ]:
packed = pack_data(raw, p=100, noise=noise,
                   trim_range=(50,800),
                   plot=True, graphing_options=gopts)

## Fit of an Exponential Decay Model to the First Data Set

Below, we provide a more compact code version of the code that you used last week for fitting a model to data. Once you have a packed data set, you can modify this code to perform a least squares fit of a model to data. In your Experiment 02 Pre-Lab, you got to try slightly altering a fitting model. Here you will replace the model entirely, in order to fit to exponential decay. 

The key pieces of code that you will need to alter are indicated by notes in Red, and include:
- the name of your data file - modify the line containing "fname = ..." 
- your fitting function - modify the line "def fit_function(....)" and "return ..."
- your initial guesses for the parameters
- the parameter names in "param_names = ..."

The final output from the code is a the best fit values for the parameters, the chi-squared value for the best fit, and a set of plots which are also stored as a JPEG. 


In [ ]:
guesses=ExponentialFitParameters(
    amplitude=0.6,
    tau=5e-4,
    offset=0.02
)

results = autofit(packed, gopts, FitType.EXPONENTIAL, guesses)

In [ ]:
results.tabulate(print_cov=True, units=('V','Ω*F', 'V'))

In [ ]:
display(results.initial_guess_graph)

In [ ]:
display(results.initial_guess_residuals_graph)

In [ ]:
display(results.autofit_graph)

In [ ]:
display(results.autofit_residuals_graph)

You should comment on the quality of the fit and record the best fit parameters and their uncertainty.

Make a rough comparison of the decay time with the expectation for an RC circuit (we will do a more quantitative comparison later).

## Measurements with Different Values of the Resistance

You can repeat this whole process with each decay curve that you have collected, and it gets easy to just run the codes on some new data.

You can copy and paste the code cells to run on additional data (don't overwrite your previous results!). You can also start storing the outputs as new variables to make it easier to compile your results and discuss them at the end.

If you would prefer, you can run the codes above with new data, or run them in a separate note book. Then record the results here, including the chi-squared values and best fit parameters with uncertainty. If you want to run the code in a separate notebook, just copy the code cells into a new notebook, or adapt the stand-alone codes that we have provided to you. Either way, you should record all fitting results in this main Expt 02 Jupyter notebook, and also include the figures for each fit. The figures can be read in and displayed using the short code below (just make sure you save each image separately with different filenames or it will be overwritten!).

For each additional resistance value, create a new header. This will make it easier for you to navigate your workbook and for the TAs to mark your work later.